## Email retrieval Code with direct output

In [ ]:
import imaplib
import email
import traceback 


# import yaml
# with open(r"C:\Users\mayur dabade\Desktop\GenAI\Mail summarizer\credentials.yml") as f:
#     content = f.read()

# my_credentials = yaml.load(content, Loader=yaml.FullLoader)
# user, password = my_credentials["user"], my_credentials["password"]

# FROM_EMAIL = user
# FROM_PWD = password


FROM_EMAIL = ""
FROM_PWD = "" 
SMTP_SERVER = "imap.gmail.com" 
SMTP_PORT = 993

from bs4 import BeautifulSoup

def read_email_from_gmail():
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(FROM_EMAIL, FROM_PWD)
        mail.select('inbox')

        data = mail.search(None, 'ALL')
        mail_ids = data[1]
        id_list = mail_ids[0].split()
        # Limit the number of emails to fetch to the latest 10
        latest_email_id = int(id_list[-1])
        first_email_id = max(1, latest_email_id - 0)  # Fetch latest 10 emails
        for i in range(latest_email_id, first_email_id - 1, -1):
            data = mail.fetch(str(i), '(RFC822)')
            for response_part in data:
                arr = response_part[0]
                if isinstance(arr, tuple):
                    msg = email.message_from_bytes(arr[1])
                    email_subject = msg['subject']
                    email_from = msg['from']
                    print('From : ' + email_from)
                    print('Subject : ' + email_subject)
                    for part in msg.walk():
                        if part.get_content_type() == "text/html":
                            # Parse HTML content using BeautifulSoup
                            soup = BeautifulSoup(part.get_payload(decode=True).decode('utf-8'), 'html.parser')
                            # Extract text from HTML and remove leading/trailing whitespace
                            email_content = "\n".join(line.strip() for line in soup.get_text(separator="\n").splitlines() if line.strip())
                            print("Content :\n" + email_content)
                            print("-" * 30)
    except Exception as e:
        traceback.print_exc()
        print(str(e))

read_email_from_gmail()


## Email retrieval Code that saves output in text file

In [ ]:
from bs4 import BeautifulSoup

def read_email_from_gmail():
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(FROM_EMAIL, FROM_PWD)
        mail.select('inbox')

        data = mail.search(None, 'ALL')
        mail_ids = data[1]
        id_list = mail_ids[0].split()
        # Limit the number of emails to fetch to the latest 10
        latest_email_id = int(id_list[-1])
        first_email_id = max(1, latest_email_id - 0)  # Fetch latest 10 emails
        with open("emails.txt", "w", encoding="utf-8") as file:
            for i in range(latest_email_id, first_email_id - 1, -1):
                data = mail.fetch(str(i), '(RFC822)')
                for response_part in data:
                    arr = response_part[0]
                    if isinstance(arr, tuple):
                        msg = email.message_from_bytes(arr[1])
                        email_subject = msg['subject']
                        email_from = msg['from']
                        file.write('From : ' + email_from + "\n")
                        file.write('Subject : ' + email_subject + "\n")
                        for part in msg.walk():
                            if part.get_content_type() == "text/html":
                                # Parse HTML content using BeautifulSoup
                                soup = BeautifulSoup(part.get_payload(decode=True).decode('utf-8'), 'html.parser')
                                # Extract text from HTML and remove leading/trailing whitespace
                                email_content = "\n".join(line.strip() for line in soup.get_text(separator="\n").splitlines() if line.strip())
                                file.write("Content :\n" + email_content + "\n")
                                file.write("-" * 30 + "\n")
    except Exception as e:
        traceback.print_exc()
        print(str(e))

read_email_from_gmail()
